In [1]:
import pickle
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import rankdata
import random

# load data

In [2]:
#  read data

print 'load data'
train, valid, test = pickle.load( open('../data/samsungQA/SA_data.pkl'))

load data


In [3]:
idx = []
for i in xrange(10000):
    idx.append(i*10)
    
test_q = test['c']
test_q = [test_q[x] for x in idx]
test_a = test['r']
test_q = [ ' '.join( [ str(x) for x in a ] ) for a in test_q ]
test_a = [ ' '.join( [ str(x) for x in a ] ) for a in test_a ]
print len(test_q)
print len(test_a)

train_q = [ ' '.join( [ str(x) for x in a ] ) for a in train['c'] ]
train_a = [ ' '.join( [ str(x) for x in a ] ) for a in train['r'] ]
print len(train_q)
print len(train_a)

10000
100000
163616
163616


## calculate tf-idf value of the corpus (train)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_corpus = []
train_corpus.extend(train_q)
train_corpus.extend(train_a)

test_corpus = []
test_corpus.extend(test_q)
test_corpus.extend(test_a)

print len(train_corpus)
print len(test_corpus)

327232
110000


In [8]:
# calculate tf-idf
tfidfv = TfidfVectorizer(ngram_range=(1,1)).fit( train_corpus )
len(tfidfv.vocabulary_)

26555

In [19]:
# tf-idf
test_tfidf_q = tfidfv.transform(test_q)
test_tfidf_a = tfidfv.transform(test_a)

test_tfidf_q_10 = []
for x in test_tfidf_q:
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)
    test_tfidf_q_10.append(x)

print np.shape(test_tfidf_q_10)
print np.shape(test_tfidf_a)
print test_tfidf_q_10[0].toarray()[0]
print test_tfidf_a[0].toarray()[0]

## calculate similarity

In [10]:
# cos-sim
from scipy.spatial.distance import cosine
cos_sim = [ cosine(x.toarray()[0], y.toarray()[0]) for x,y in zip(test_tfidf_q_10, test_tfidf_a) ]
len(cos_sim)

100000

In [11]:
cos_sim_10 = np.reshape(cos_sim, [10000, 10])
print len(cos_sim_10)
print cos_sim_10[0]
print rankdata(cos_sim_10[0], method='min')

10000
[ 0.61844372  0.98767536  0.98485391  0.89969295  0.99574509  0.99677441
  0.98153741  0.98867731  0.98259301  0.99755754]
[ 1  6  5  2  8  9  3  7  4 10]


### 1 in 10 case

In [12]:
rank = [ rankdata(x, method='min') for x in cos_sim_10 ]
print rank[1]

# 1 in 10
print sum([1 for x in rank if x[0] <= 1]) / float(10000)
print sum([1 for x in rank if x[0] <= 2]) / float(10000)
print sum([1 for x in rank if x[0] <= 5]) / float(10000)

[ 1  9  5  7  2  4  6  3  8 10]
0.8346
0.8974
0.9539


### 1 in 2 case

In [17]:
sum = 0
for x in cos_sim_10:
    y_true = x[0]
    y_false = random.choice(x[1:])
    if y_true < y_false:
        sum += 1
sum / float(10000)        

0.9376